In [1]:
'''
In the following we will explore a number of techniques, including padding and strided convolutions, 
that offer more control over the size of the output. As motivation, note that since kernels generally have width and height greater than 1, 
after applying many successive convolutions, we tend to wind up with outputs that are considerably smaller than our input. If we start with a 240*240 pixel image, 
ten layers of 5*5 convolutions reduce the image to 200*200 pixels, slicing off 30% of the image and with it obliterating any interesting information on the boundaries of the original image. 
Padding is the most popular tool for handling this issue. In other cases, we may want to reduce the dimensionality drastically, e.g.,
if we find the original input resolution to be unwieldy. Strided convolutions are a popular technique that can help in these instances.
'''
import torch
from torch import nn

In [3]:
'''
卷积神经网络通常使用具有奇数高度和宽度值的卷积核，例如 1，3，5 或 7。选择奇数尺寸的卷积核有如下优势：

1. **对称性**：奇数尺寸卷积核允许在空间上保持对称性。每个维度的卷积核大小使我们可以在上下方向上添加相同数量的行，并在左右方向上添加相同数量的列。因此，卷积核的中心像素正好位于相邻像素的中心，这有助于保持输入和输出特征图之间的空间关系。

2. **保持尺寸不变**：使用奇数尺寸的卷积核，我们可以通过添加与卷积核大小相适应的填充，轻松地保持输入和输出特征图的空间维度不变。例如，在应用 3x3 的卷积核时，我们可以在输入图像的边界周围添加一个像素的填充，以确保输出特征图与输入具有相同的尺寸。

3. **局部连接**：奇数尺寸的卷积核有助于捕获输入图像中局部区域的信息。由于以中心像素为基准建立卷积核，这简化了捕捉局部信息和结构的过程。

综上所述，在卷积神经网络中使用奇数尺寸的卷积核有助于保持输入和输出特征图之间的空间关系，并允许网络更好地捕获和保留局部图像信息。 
'''
# We define a helper function to calculate convolutions. It initializes the
# convolutional layer weights and performs corresponding dimensionality
# elevations and reductions on the input and output


def comp_conv2d(conv2d, X):
    # (1, 1) indicates that batch size and the number of channels are both 1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # Strip the first two dimensions: examples and channels
    return Y.reshape(Y.shape[2:])


# 1 row and column is padded on either side, so a total of 2 rows or columns
# are added
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([8, 8])

In [4]:
# We use a convolution kernel with height 5 and width 3. The padding on either
# side of the height and width are 2 and 1, respectively
conv2d = nn.LazyConv2d(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [5]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [6]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

1. Given the final code example in this section with kernel size (3,5), padding (0,1), and stride (3,4), calculate the output shape to check if it is consistent with the experimental result.

2. For audio signals, what does a stride of 2 correspond to?

3. Implement mirror padding, i.e., padding where the border values are simply mirrored to extend tensors.

4. What are the computational benefits of a stride larger than 1?

5. What might be statistical benefits of a stride larger than 1?

6. How would you implement a stride of 1/2? What does it correspond to? When would this be useful?


1. 使用给定的最终示例中的卷积核大小（3,5），填充（0,1）和步幅（3,4），计算输出形状，检查它是否与实验结果一致。

假设我们的输入图像尺寸为（C_in, H_in, W_in），即（通道数，高度，宽度）。在本例中，我们有以下参数：
   - 卷积核大小：(3, 5)
   - 填充：(0, 1)
   - 步幅：(3, 4)

输出形状的计算方法为：

   - 输出图像高度：H_out = (H_in - Kernel_H + 2*Padding_H) / Stride_H + 1
   - 输出图像宽度：W_out = (W_in - Kernel_W + 2*Padding_W) / Stride_W + 1

请注意，实际的输入尺寸未给出，因此计算输出形状需要具体的输入尺寸。

2. 对于音频信号，步幅为2对应于什么？

   对于音频信号，步幅大小代表着在处理音频信号时移动的单位时间长度。一个步幅为2意味着在从输入音频序列到输出序列的转换过程中，每次移动跳过一个单位（例如采样点）以处理相邻的音频片段。这就意味着较高的计算效率，但可能会导致信号中的一些信息丢失。

3. 实现镜像填充，即填充边界值以扩展张量的方法是简单地将其镜像。

```python
import torch.nn.functional as F

def mirror_padding(x, padding):
    x = F.pad(x, padding, mode='reflect')
    return x
```

4. 步幅大于1的计算优势是什么？

   步幅大于1的计算优势包括：
   - 更快的计算：较大的步幅意味着在处理数据时，需要计算的局部区域更少。
   - 更小的输出尺寸：较大的步幅导致较小的输出尺寸，这使模型需要处理较少的参数。

5. 步幅大于1的统计优势是什么？

   步幅大于1的统计优势包括：
   - 抽象程度更高：较大的步幅导致较小的输出尺寸和更高级别的特征表示。
   - 视野范围更广：较大的步幅使得在更大的输入区域内进行卷积核操作。

6. 如何实现步幅为1/2的方法？这对应于什么？这何时有用？

   实现步幅为1/2的方法是使用上采样（上卷积）操作。上采样（或反卷积）用于在空间分辨率方面扩展特征映射。步幅为1/2意味着输出的宽度和高度是输入的一半。以下是实现步幅为1/2的示例：

```python
import torch.nn as nn

class UpsamplingLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(UpsamplingLayer, self).__init__()
        self.conv_transpose = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)

    def forward(self, x):
        return self.conv_transpose(x)
```

   使用步幅为1/2的反卷积特别有用于图像生成任务中，例如图像超分辨率和生成对抗网络（GANs）。